# **Proyecto 1 - Etapa I**

Integrantes:
* Andrés Francisco Borda - 201729184
* Juan Pablo Lora Hernández - 202012524
* Gabriela Vargas Rojas - 202013830


# 1. Entendimiento de los datos

In [3]:
#Instalaciones

#Libreria para remplazar numeros por palabras
!pip install num2words
#Instalar  pandas profiler
!pip install pandas-profiling==2.7.1
#deteccion de lenguaje para eliminar entradas que no esten en español
!pip install langdetect
#Procesamientno de lenguaje natural en español
!pip install stanza

You should consider upgrading via the 'C:\Users\WIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ERROR: Could not find a version that satisfies the requirement pandas-profiling==2.7.1 (from versions: 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 3.6.1, 3.6.2, 3.6.3, 3.6.4, 3.6.5, 3.6.6)
ERROR: No matching distribution found for pandas-profiling==2.7.1
You should consider upgrading via the 'C:\Users\WIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\WIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\WIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
#imports para procesamiento de texto

#tokenizacion y lematizacion
import stanza
#Para integrar pasos de la limpieza adicionales
from stanza.pipeline.processor import Processor, register_processor
#paquete español
stanza.download('es')

#Para manejo de numeros, singluares, plurarles en lenguaje
from num2words import num2words
#Deteccion de lenguaje
from langdetect import detect
# librería Natural Language Toolkit, usada para trabajar con textos
import nltk
# Punkt permite separar un texto en frases.
nltk.download('stopwords')
from nltk.corpus import stopwords

#Operaciones con expresiones regulares y unicode
import re, string, unicodedata

c:\Users\WIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-14 14:03:24 INFO: Downloading default packages for language: es (Spanish) ...
2023-10-14 14:03:25 INFO: File exists: C:\Users\WIN\stanza_resources\es\default.zip
2023-10-14 14:03:29 INFO: Finished downloading models and saved to C:\Users\WIN\stanza_resources.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#Imports generales para analisis de datos y ML
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
import statistics

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

c:\Users\WIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\WIN\AppData\Local\Programs\Python\Python310\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\WIN\AppData\Local\Temp\ipykernel_12536\122007551.py:5: DeprecationWarning: `import p

# Entendimiento de los datos

In [6]:
# Lectura de los datos.
df_ODS = pd.read_excel("cat_6716.xlsx")


df_ODS.head()


,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequí...",6
1,Hay una gran cantidad de literatura sobre Aust...,6
2,"Los procesos de descentralización, emprendidos...",6
3,Esto puede tener consecuencias sustanciales pa...,6
4,La función de beneficio también incorpora pará...,6


In [7]:
df_ODS.dtypes

Textos_espanol    object
sdg                int64
dtype: object

#Imprimimos reporte de pandas Profiler

In [8]:
mensajes = df_ODS.copy()
#Longitud del comentario (numero de caracteres)
mensajes['Conteo'] = [len(x) for x in mensajes['Textos_espanol']]
#Palabra(s) mas frecuente(s)
mensajes['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in mensajes['Textos_espanol']]
#Longitud de la palabra mas larga del comentario
mensajes['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in mensajes['Textos_espanol']]
#Longitud de la palabra mas corta del comentario
mensajes['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in mensajes['Textos_espanol']]

ProfileReport(mensajes)


TypeCheckError: argument "config_file" (None) did not match any element in the union:
  pathlib.Path: is not an instance of pathlib.Path
  str: is not an instance of str

In [9]:
#Conteo de entradas nulas
df_ODS.isnull().sum()

Textos_espanol    0
sdg               0
dtype: int64

In [10]:
#verificamos entradas en otros idiomas
df_ODS[df_ODS.Textos_espanol.map(detect)  != 'es']

,Textos_espanol,sdg
1317,"Project Finance NewsWire, Discussion with four...",7
1553,Innovación en tecnología energética: Aprender ...,7
1950,Empleos verdes y energías renovables: Low carb...,7


In [11]:
#Revisamos los registros duplicados
df_ODS.duplicated().sum()

0

# 2. Preparacion de los datos

In [12]:
#Eliminamos Duplicados
df_ODS = df_ODS.drop_duplicates()
#Eliminamos entradas en otros idiomas
df_ODS = df_ODS.drop(df_ODS[df_ODS.Textos_espanol.map(detect)  != 'es'].index)

In [13]:
 #verificamos cuantos datos quedan
df_ODS.shape

(2996, 2)

## 2.1 Limpieza, tokenizacion y lemmatizacion

In [14]:
#Remplaza los numeros por su representacion en palabras
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
#Remueve todo caracter no latino (conserva espacios y numeros)
def remove_nonlatin(words):
  new_words = []
  for word in words:
    new_word = ''
    for ch in word:
      if unicodedata.name(ch).startswith(('LATIN', 'DIGIT', 'SPACE')):
        new_word += ch
    new_words.append(new_word)
  return new_words

#Remueve palabras comunes que no aportan informacion
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    s = set(stopwords.words('spanish'))
    for word in words:
        if word not in s:
            new_words.append(word)
    return new_words

#Remueve puntuacion
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = ''
    for word in words:
            new_words += re.sub(r'[^\w\s]', ' ', word)
    return new_words

 #Procesamiento de cada review usando stanza
def tokenLemma():
  df_ODS['words'] = df_ODS['Textos_espanol'].apply(remove_punctuation)
  #Creamos un pipeline para tokenizacion y lematizacion
  nlp = stanza.Pipeline('es', processors = 'tokenize,mwt,pos,lemma', use_gpu=True)
  in_docs = [stanza.Document([], text=d) for d in df_ODS.words]
  return nlp(in_docs)

#Funcion secundaria para procesar cada token
def procesamientoPalabras(words):
    words = remove_nonlatin(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#Funcion principal para el pre-procesamiento
def preprocessing():
  out_docs = tokenLemma()
  palabras = []

  for doc in out_docs:
    reviewAct = []
    for sentence in doc.sentences:
      for word in sentence.words:
        if(word.pos != 'PUNCT' and word.pos != 'SYM'):
          reviewAct.append(word.lemma.lower())
    palabras.append(reviewAct)

  df_ODS['words'] = palabras
  df_ODS['words'] = df_ODS['words'].apply(procesamientoPalabras)

In [15]:
#Descomentar el processing si se quiere revisar, si no el resultado con los datos procesados esta en el archivo df_procesado
#preprocessing()


In [16]:
# Cargamos los datos procesados

#Guardamos los datos procesados. Descomentar si se quiere crear el archivo despues de procesar los datos.
#df_ODS.to_csv('df_procesado.csv', index=False, sep= ";")

#Leemos los datos procesados
df_ODS_Pre = pd.read_csv("df_procesado.csv", sep = ";", encoding='utf-8')
import ast
df_ODS_Pre['words'] = df_ODS_Pre['words'].apply(ast.literal_eval)



In [17]:
#Visualizar los cambios
df_ODS_Pre.head()

,Textos_espanol,sdg,words
0,"Es importante destacar que, en un año de sequí...",6,"[ser, importante, destacar, año, sequía, esper..."
1,Hay una gran cantidad de literatura sobre Aust...,6,"[haber, gran, cantidad, literatura, australia,..."
2,"Los procesos de descentralización, emprendidos...",6,"[proceso, descentralización, emprendido, serie..."
3,Esto puede tener consecuencias sustanciales pa...,6,"[poder, tener, consecuencia, sustancial, calid..."
4,La función de beneficio también incorpora pará...,6,"[función, beneficio, incorporar, parámetro, af..."


In [18]:
#Convertimos los tokens nuevamente en strings
df_ODS_Pre['words'] = df_ODS_Pre['words'].apply(lambda x: ' '.join(map(str, x)))
df_ODS_Pre

,Textos_espanol,sdg,words
0,"Es importante destacar que, en un año de sequí...",6,ser importante destacar año sequía esperar dis...
1,Hay una gran cantidad de literatura sobre Aust...,6,haber gran cantidad literatura australia área ...
2,"Los procesos de descentralización, emprendidos...",6,proceso descentralización emprendido serie act...
3,Esto puede tener consecuencias sustanciales pa...,6,poder tener consecuencia sustancial calidad ag...
4,La función de beneficio también incorpora pará...,6,función beneficio incorporar parámetro afectar...
...,...,...,...
2992,"""Forjando una Convención para los Crímenes con...",16,forjar convención crímenes humanidad incluir t...
2993,"En 2016, Daragh Murray publicó su libro Obliga...",16,dos mil dieciséis daragh murray publicar libro...
2994,"En este libro, el autor argumenta que el activ...",16,libro autor argumentar activismo judicial diri...
2995,"* Facultad de Derecho, Universidad de Manchest...",16,facultad derecho universidad manchester sw v s...


### 2.1.1 Verificamos los datos obtenidos

In [19]:
mensajes = df_ODS_Pre.copy()
#Longitud del comentario (numero de caracteres)
mensajes['Conteo'] = [len(x) for x in mensajes['words']]
#Palabra(s) mas frecuente(s)
mensajes['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in mensajes['words']]
#Longitud de la palabra mas larga del comentario
mensajes['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in mensajes['words']]
#Longitu de la palabra mas corta del comentario
mensajes['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in mensajes['words']]

ProfileReport(mensajes)

TypeCheckError: argument "config_file" (None) did not match any element in the union:
  pathlib.Path: is not an instance of pathlib.Path
  str: is not an instance of str

## 2.2 Separacion de los datos y vectorización

In [20]:
#Separación de los datos en conjunto de test y train
df_ODS_Pre = df_ODS_Pre.drop('Textos_espanol', axis = 1)
df_train, df_test = sklearn.model_selection.train_test_split(df_ODS_Pre, test_size=0.2, random_state=0)

X_train = df_train['words']
y_train = df_train['sdg']

X_test = df_test['words']
y_test = df_test['sdg']

In [21]:
#Vectorizar los datos con Tfid
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(2397, 12505) (600, 12505)


# 3. Modelado y evaluacion

## 3.1 Random Forest - Gabriela Vargas

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clasif = RandomForestClassifier(random_state=0)
clasif.fit(train_vectors, y_train)
ytest_pred = clasif.predict(test_vectors)

clasif = RandomForestClassifier(random_state=0)
clasif.fit(train_vectors, y_train)
predicted = clasif.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)


Classification Report
              precision    recall  f1-score   support

           6       0.98      0.97      0.97       213
           7       0.98      0.96      0.97       200
          16       0.97      1.00      0.99       187

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [23]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#Se fijan 5 particiones teniendo en cuenta los datos que estamos manejando.
particiones = KFold(n_splits=5, shuffle=True, random_state = 0)

# Establecemos el espacio de búsqueda para los hiperparámetros en especial los que nos permiten controlar la complejidad y el rendimiento del modelo
param_grid = {'criterion':['gini', 'entropy'],'min_samples_split':[2, 4, 6],'max_features':[10, 100, 1000]}

forest = RandomForestClassifier(random_state=0)
modelo = GridSearchCV(forest, param_grid, cv=particiones, refit = True, scoring = 'accuracy')
modelo.fit(train_vectors, y_train)

print(modelo.best_params_)  #MMejor modelo escogido

clasif = modelo.best_estimator_
predicted = clasif.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted)

0.97

Teniendo en cuenta el accuaracy score, nos pudimos dar cuenta que el Random Forest está realizando buenas predicciones con los datos que tenemos, lo cual nos puede indicar que este modelo nos permite relacionar de manera automática un texto según los ODS de manera casi precisa.

## 3.2 Logistic Regression - Andrés Borda

In [24]:
from matplotlib import style
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [25]:

from sklearn.model_selection import GridSearchCV

# parameter grid para evaluar el mejor modelo
parameters = {
    'penalty' : ['l1','l2', 'elasticnet', None], 
    'C'       : np.logspace(-10,10,3),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}
     

In [26]:
#Implementar la regresin con k-fold cross validation
from sklearn.model_selection import RepeatedKFold
metrica = RepeatedKFold(n_splits=20, n_repeats= 10, random_state=0)

logreg = LogisticRegression()
modelo = GridSearchCV(logreg, param_grid = parameters, scoring='accuracy', cv=metrica, n_jobs=-1)  

In [27]:
modelo.fit(train_vectors,y_train)
print("Tuned Hyperparameters :", modelo.best_params_)
print("Accuracy :",modelo.best_score_)
modelo_optimo = modelo.best_estimator_

KeyboardInterrupt: 

In [ ]:
#Medimos exatitud sobre el test
prediccion = modelo.predict(test_vectors)
score = modelo.score(test_vectors, y_test)

print("Exactitud predicción test: " + str(score))

Exactitud predicción test: 0.9866666666666667


In [ ]:
#Encontramos la matriz de confusion con los resultados
matriz_confusion = metrics.confusion_matrix(y_test, prediccion)
print(matriz_confusion)

[[208   3   2]
 [  3 197   0]
 [  0   0 187]]


In [ ]:
# Reporte de metricas
reporte = classification_report(y_test, prediccion)
print(reporte)

              precision    recall  f1-score   support

           6       0.99      0.98      0.98       213
           7       0.98      0.98      0.98       200
          16       0.99      1.00      0.99       187

    accuracy                           0.99       600
   macro avg       0.99      0.99      0.99       600
weighted avg       0.99      0.99      0.99       600



## 3.3 MultinomialNB - Juan Pablo Lora

In [29]:
from sklearn.naive_bayes import MultinomialNB
multinomial = MultinomialNB().fit(train_vectors, y_train)

In [30]:
y_pred = multinomial.predict(test_vectors)

In [31]:
print("Score: ", multinomial.score(train_vectors, y_train))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

Score:  0.9933249895702962
Accuracy: 0.9766666666666667
F1 score: 0.9770658067203936


In [32]:
confusion_matrix(y_test, y_pred)

array([[205,   6,   2],
       [  4, 195,   1],
       [  1,   0, 186]], dtype=int64)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           6       0.98      0.96      0.97       213
           7       0.97      0.97      0.97       200
          16       0.98      0.99      0.99       187

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600

